In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import numpy as np
#sns.set_style('darkgrid')
sns.set(font_scale=3,font="simhei")

#--- 改变字体
font = {'weight' : 'bold',
        'size'   : 22}
matplotlib.rc('font', **font)

#--- plt设置
SMALL_SIZE = 16
MEDIUM_SIZE = 18
BIGGER_SIZE = 22

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [12]:
import re
re_X = re.compile('\(([0-9]*), ([0-9]*)\)')
re_find_k = re.compile('output([0-9]+)_.*\.txt')
TOL = 1e-4

def Parse_X(m, n, X):
    sent = [0] * m
    received = [0] * n
    for road, amount in X.items():
        re_X_ret = re_X.match(road)
        i, j = int(re_X_ret[1]), int(re_X_ret[2])
        sent[i] += amount
        received[j] += amount
    return sent, received

def Analyze_Simulation(m, n, I, simulations):
    """
    simulations: Dict[str, Dict[str, Any]]
    """
    rets = []
    for cnt, simulation in simulations.items():
        d_r = simulation['d_r']
        X = np.asarray(simulation['X'])

        sent, received = X.sum(axis=1), X.sum(axis=0)

        # metrics:
        # 1. supply: utilization_rate,
        # 2. demand: fulfill_rate, event_fulfill_rate, unmet_amount,
        utilization_rate = sum(sent) / sum(I)
        fulfill_rate = sum(received) / sum(d_r) if sum(d_r) != 0 else 1  # sum(d_r) possibly is zero
        event_fulfill_rate = sum([abs(r - d) <= TOL for r, d in zip(received, d_r)]) / n
        unmet_demand = sum(d_r) - sum(received)

        # construct return
        ret = {'utilization_rate': utilization_rate,
               'fulfill_rate': fulfill_rate,
               'event_fulfill_rate': event_fulfill_rate,
               'unmet_demand': unmet_demand,
               'cnt': cnt}

        rets.append(ret)
    return rets

In [18]:
import json

with open('./saa.txt') as f:
    line = f.readline()
    saa = json.loads(line)

with open('./co_CI=0_rep=10000.txt') as f:
    line = f.readline()
    co = json.loads(line)

with open('./mv_CI=0_rep=10000.txt') as f:
    line = f.readline()
    mv = json.loads(line)

FileNotFoundError: [Errno 2] No such file or directory: './co_CI=0_rep=10000.txt'

In [31]:
co_pd = pd.DataFrame(Analyze_Simulation(7, 23,co['sol']['I'], co['simulation_outsample']))
co_pd

,utilization_rate,fulfill_rate,event_fulfill_rate,unmet_demand,cnt
0,1.000000,0.932745,0.608696,60.52942,0
1,0.327587,1.000000,1.000000,0.00000,1
2,0.537243,1.000000,1.000000,0.00000,2
3,0.382384,1.000000,1.000000,0.00000,3
4,0.545582,1.000000,1.000000,0.00000,4
5,0.577745,1.000000,1.000000,0.00000,5
6,0.625394,1.000000,1.000000,0.00000,6
7,0.468152,1.000000,1.000000,0.00000,7
8,0.334735,1.000000,1.000000,0.00000,8
9,0.250158,1.000000,1.000000,0.00000,9


In [33]:
mv_pd = pd.DataFrame(Analyze_Simulation(7, 23, mv['sol']['I'], mv['simulation_outsample']))
mv_pd

,utilization_rate,fulfill_rate,event_fulfill_rate,unmet_demand,cnt
0,1.000000,0.987067,0.956522,11.6396,0
1,0.309559,1.000000,1.000000,0.0000,1
2,0.507677,1.000000,1.000000,0.0000,2
3,0.361340,1.000000,1.000000,0.0000,3
4,0.515557,1.000000,1.000000,0.0000,4
5,0.545950,1.000000,1.000000,0.0000,5
6,0.590976,1.000000,1.000000,0.0000,6
7,0.442388,1.000000,1.000000,0.0000,7
8,0.316313,1.000000,1.000000,0.0000,8
9,0.236391,1.000000,1.000000,0.0000,9


In [20]:
saa_pd = pd.DataFrame(Analyze_Simulation(7, 23, saa['sol']['I'], saa['simulation_outsample']))
saa_pd

,utilization_rate,fulfill_rate,event_fulfill_rate,unmet_demand,cnt
0,0.456,0.506667,0.73913,444.0,0
1,0.275,1.000000,1.00000,0.0,1
2,0.451,1.000000,1.00000,0.0,2
3,0.321,1.000000,1.00000,0.0,3
4,0.458,1.000000,1.00000,0.0,4
5,0.485,1.000000,1.00000,0.0,5
6,0.525,1.000000,1.00000,0.0,6
7,0.393,1.000000,1.00000,0.0,7
8,0.281,1.000000,1.00000,0.0,8
9,0.210,1.000000,1.00000,0.0,9


In [21]:
saa['sol']

{'I': [225.0, 0.0, 0.0, 0.0, 0.0, 136.0, 639.0],
 'Z': [1.0, 0.0, -0.0, -0.0, -0.0, 1.0, 1.0],
 'obj': 75.71947122619991,
 'h': 68.012829431,
 'f': 7.7066417952}